In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import xlrd
from itertools import product
from sys import stdout as out
from mip import Model, xsum, minimize, BINARY

In [2]:
file_path = 'data.xlsx'
df = pd.read_excel(file_path)
wb = xlrd.open_workbook(file_path) 
d = np.zeros((30,30))
p = np.zeros((30,30))
xc = np.zeros((4))
sheet_d = wb.sheet_by_index(0) 
sheet_p = wb.sheet_by_index(1)
for i in range(30):
    for j in range(30):
        d[i][j] = sheet_d.cell_value(i, j)
        p[i][j] = sheet_p.cell_value(i, j)
        

In [3]:
# number of nodes and list of vertices
n, V = len(d), set(range(len(d)))

places = list(range(1, n+1))

In [4]:
c = d

In [5]:
for i in range(30):
    for j in range(30):
        if p[i][j] >= 0.6:
            c[i][j] = 1000
            

In [6]:
model = Model()

# binary variables indicating if arc (i,j) is used on the route or not
x = [[model.add_var(var_type=BINARY) for j in V] for i in V]

# continuous variable to prevent subtours: each city will have a
# different sequential id in the planned route except the first one
y = [model.add_var() for i in V]

# objective function: minimize the distance
model.objective = minimize(xsum(c[i][j]*x[i][j] for i in V for j in V))

# constraint : leave each city only once
for i in V:
    model += xsum(x[i][j] for j in V - {i}) == 1

# constraint : enter each city only once
for i in V:
    model += xsum(x[j][i] for j in V - {i}) == 1

# subtour elimination
for (i, j) in product(V - {0}, V - {0}):
    if i != j:
        model += y[i] - (n+1)*x[i][j] >= y[j]-n

# optimizing
model.optimize()

# checking if a solution was found
if model.num_solutions:
    out.write('route with total distance %g found: %s'
              % (model.objective_value, places[0]))
    nc = 0
    while True:
        nc = [i for i in V if x[nc][i].x >= 0.99][0]
        out.write(' -> %s' % places[nc])
        if nc == 0:
            break
    out.write('\n')

# sanity tests
from mip import OptimizationStatus
assert model.status == OptimizationStatus.OPTIMAL
model.check_optimization_results()

route with total distance 1115 found: 1 -> 21 -> 11 -> 19 -> 26 -> 10 -> 2 -> 24 -> 29 -> 8 -> 7 -> 15 -> 3 -> 4 -> 23 -> 20 -> 5 -> 6 -> 28 -> 12 -> 16 -> 22 -> 18 -> 13 -> 9 -> 27 -> 25 -> 14 -> 17 -> 30 -> 1
